# Tech Stack
I decided to use **Docker** to run local services for a **Postgres database**, **dbt**, and **Jupyter notebook**. My reasoning is that I already had this infrastructure set up for my own projects so I could quickly assemble it and start the exercise.

I pushed my code to this git repo: https://github.com/danbratton/abnormal_exercise.

Instructions to run the repo locally are in the [README.md](https://github.com/danbratton/abnormal_exercise/blob/main/dbt/README.md). 

# Loading Data
I decided to load the data into Postgres using dbt **seeds**. Although seeds are for slowly changing or static data, it was quick and easy to use for this exercise. I put the CSV files in the `seeds` directory and then ran `dbt seed`.

# Running SQL
I developed my SQL in **DBeaver** and put the final SQL below, which I execute with `psycopg2`. I used `pandas` to display the results in a nice format.

In [1]:
import psycopg2
import pandas as pd
import os


def run_select(sql):
    # helper function for executing queries and displaying results in a nice format
    # note: the environment variables are set by compose.yaml from the .env-local file
    conn = psycopg2.connect(
        database=os.environ['DEV_PG_NAME'], 
        user=os.environ['DEV_PG_USER'],  
        password=os.environ['DEV_PG_PASSWORD'], 
        host=os.environ['DEV_PG_HOST'], 
        port=os.environ['DEV_PG_PORT']
    )
    cursor = conn.cursor()
    cursor.execute(sql) 
    results = cursor.fetchall() 
    cols = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(results, columns=cols)
    conn.commit() 
    conn.close() 
    return df


# Exercise

## 1.a. How many customers do we have ATO product data for?
* I'm assuming a customer is identified by the `customer_id`
* I am NOT going to assume all the customers with ATO data are in the `customers` table. That is why I counted all the unique `customer_id`s across all event tables instead of counting the customers in the `customers` table.

In [2]:
sql = """
WITH customer_ids AS (
SELECT
	customer_id
FROM
	user_events
UNION
SELECT
	customer_id
FROM
	detector_events de
UNION
SELECT
	customer_id
FROM
	case_events ce
UNION
SELECT
	customer_id
FROM
	customer_case_events cce
UNION
SELECT
	customer_id
FROM
	customer_action_events cae
)

SELECT 
	count(customer_id) AS customer_count
FROM
	customer_ids
WHERE
	customer_id IS NOT NULL
"""
run_select(sql)

,customer_count
0,10


## 1.b. Over what time period does the ATO data exist?

In [3]:
sql = """
WITH all_events_timestamps AS (
SELECT
		event_timestamp AS timestamp
FROM
	user_events
UNION
SELECT
		event_timestamp AS timestamp
FROM
	detector_events de
UNION
-- I'm assuming we care about the event timestamp rather than the customer-reported event timestamp
SELECT
		reported_event_timestamp AS timestamp
FROM
	customer_case_events cce
UNION
SELECT
		action_timestamp AS timestamp
FROM
	case_action_events cae
UNION
SELECT 
		customer_action_ts AS timestamp
FROM
	customer_action_events cae2
)

SELECT 
	min(timestamp) AS data_date_from,
	max(timestamp) AS data_date_to
FROM
	all_events_timestamps
WHERE
	timestamp IS NOT NULL
"""
run_select(sql)

,data_date_from,data_date_to
0,2020-01-01 03:56:31,2021-01-03 15:20:10


## 1.c. How many ATO detectors are there?
* I'm assuming the detectors are identified by the `detector_id`

In [4]:
sql = """
SELECT 
	count(DISTINCT(detector_id)) AS detector_count
FROM
	detector_events de
"""
run_select(sql)

,detector_count
0,6


## 1.d. What is the range of customer sizes by number of users?
* I'm assuming `cnt_users` is the count of the number of users in a company.

In [5]:
sql = """
SELECT 
	min(cnt_users) AS company_user_count_from,
	max(cnt_users) AS company_user_count_to
FROM
	customers c
"""
run_select(sql)

,company_user_count_from,company_user_count_to
0,14,371


## 2.a. What is the average monthly false negative rate of the ATO product?
* False negative rate = false negatives / (true positives + false negatives)
* False negatives occur when the customer reports an ATO event that was not detected by Abnormal. Meaning, cases in `customer_case_events` are false negatives.

In [6]:
sql = """
WITH false_negatives AS (
SELECT
	-- using the customer-provided actual time the event happened to align with detector_events.event_timestamp
	date_trunc('month',
	reported_event_timestamp) AS MONTH,
		count(reported_case_id) AS false_negatives
FROM
	customer_case_events cce
GROUP BY
	MONTH
)

,
true_positives AS (
SELECT
		date_trunc('month',
	de.event_timestamp) AS MONTH,
		count(*) AS true_positives
FROM
	case_events ce
LEFT JOIN customer_action_events cae ON
	ce.case_id = cae.case_id
LEFT JOIN detector_events de ON
	ce.triggering_event_id = de.event_id
WHERE
	customer_action_type = 'RESOLVE'
GROUP BY
	MONTH
)

,
monthly_false_negative_rate AS (
SELECT
	COALESCE (fn.month,
	tp.month) AS MONTH,
	COALESCE (false_negatives,
	0) AS false_negatives,
	COALESCE (true_positives,
	0) AS true_positives,
	CASE
		WHEN COALESCE (false_negatives,
		0) + COALESCE (true_positives,
		0) = 0 THEN NULL
		WHEN COALESCE (false_negatives,
		0) = 0 THEN 0
		ELSE round(COALESCE (false_negatives,
		0)::NUMERIC /(COALESCE (false_negatives,
		0) + COALESCE (true_positives,
		0))* 100,
		4)
	END AS false_negative_rate
FROM
	false_negatives fn
	-- full join so months without FNs or TPs are not excluded in the join
FULL JOIN true_positives tp ON
	fn.month = tp.month
)

 SELECT
	round(avg(false_negative_rate),
	2) AS average_monthly_false_negative_rate_pct
FROM
	monthly_false_negative_rate
"""
run_select(sql)

,average_monthly_false_negative_rate_pct
0,12.50


## 2.b. What is the average monthly false positive rate of the ATO product?
* False positive rate = False positives / (False positives + true negatives)
* False positives occur when Abnormal flags a breach that the customer reports as not a threat
* True negatives are events that (1) did not trigger a case and (2) were not flagged by customers as false negatives

In [7]:
sql="""
WITH true_negatives AS (
SELECT
		date_trunc('month',
	ue.event_timestamp) AS MONTH,
		count(*) AS true_negatives
FROM
	user_events ue
	-- exclude events reported by customers as a false negative
WHERE
	NOT EXISTS (
	SELECT
		reported_event_id
	FROM
		customer_case_events cce
	WHERE
		ue.event_id = cce.reported_event_id)
	-- exclude events that triggered a case
	AND NOT EXISTS (
	SELECT
		triggering_event_id
	FROM
		case_events ce
	WHERE
		ue.event_id = ce.triggering_event_id)
	AND ue.event_timestamp IS NOT NULL
GROUP BY
	MONTH
)

,
false_positives AS (
SELECT
		date_trunc('month',
	de.event_timestamp) AS MONTH,
		count(*) AS false_positives
FROM
	case_events ce
LEFT JOIN customer_action_events cae ON
	ce.case_id = cae.case_id
LEFT JOIN detector_events de ON
	ce.triggering_event_id = de.event_id
WHERE
	customer_action_type = 'NOT_A_COMPROMISE'
GROUP BY
	MONTH
)

,
monthly_false_positive_rate AS (
SELECT
	COALESCE (tn.month,
	fp.month) AS MONTH,
	COALESCE (true_negatives,
	0) AS true_negatives,
	COALESCE (false_positives,
	0) AS false_positives,
	CASE
		WHEN COALESCE (true_negatives,
		0) + COALESCE (false_positives,
		0) = 0 THEN NULL
		WHEN COALESCE (false_positives,
		0) = 0 THEN 0
		ELSE round(COALESCE (false_positives,
		0)::NUMERIC / (COALESCE (false_positives,
		0) + COALESCE (true_negatives,
		0)) ,
		4)* 100
	END AS false_positive_rate_pct
FROM
	true_negatives tn
FULL JOIN false_positives fp ON
	tn.month = fp.month
)


SELECT
	round(avg(false_positive_rate_pct),
	2) AS monthly_average_false_positive_rate_pct
FROM
	monthly_false_positive_rate
"""
run_select(sql)

,monthly_average_false_positive_rate_pct
0,0.36


## 2.c. What fraction of user events are flagged by a detector at any confidence? 

In [8]:
sql = """
WITH flagged_user_events AS (
SELECT 
		ue.event_id,
		CASE
			WHEN de.event_id IS NOT NULL THEN 1
		ELSE 0
	END AS flagged_by_detector
FROM
	user_events ue
LEFT JOIN detector_events de ON
	ue.event_id = de.event_id
)

SELECT
	round(sum(flagged_by_detector)/ count(*)::NUMERIC,
	4)* 100 AS flagged_user_events_pct
FROM
	flagged_user_events
"""
run_select(sql)

,flagged_user_events_pct
0,17.5400


## At high confidence?

In [9]:
sql = """
WITH flagged_user_events AS (
SELECT 
		ue.event_id,
		confidence_level,
		CASE
			WHEN de.event_id IS NOT NULL
		AND de.confidence_level = 'HIGH' THEN 1
		ELSE 0
	END AS flagged_by_detector
FROM
	user_events ue
LEFT JOIN detector_events de ON
	ue.event_id = de.event_id
)

SELECT
	round(sum(flagged_by_detector)/ count(*)::NUMERIC,
	4)* 100 AS flagged_user_events_pct
FROM
	flagged_user_events
"""
run_select(sql)

,flagged_user_events_pct
0,8.0000


## 2.d. Calculate the precision of each detector based on cases surfaced to customers.
* I am assuming 'precision' means a measurement of the portion of cases flagged by detectors that were true positives
* Precision = true positives / (true_positives + false_positives)
* True positives are when a case is opened by a detector which a customer resolves with `customer_action_type = 'RESOLVE'`
* False positives are when a case is opened by a detector which a customer resolves with `customer_action_type = 'NOT_A_COMPROMISE'`

In [10]:
# I'm going to use these CTEs to answer the next few questions
ctes = """
WITH true_positives AS (
SELECT 
		de.detector_id,
		count(cae.case_id) AS true_positives
FROM
	customer_action_events cae
LEFT JOIN case_events ce ON
	cae.case_id = ce.case_id
LEFT JOIN detector_events de ON
	ce.triggering_event_id = de.event_id
WHERE
	cae.customer_action_type = 'RESOLVE'
GROUP BY
	detector_id
)

,
false_positives AS (
    SELECT 
    		de.detector_id,
    		count(cae.case_id) AS false_positives
    FROM
    	customer_action_events cae
    LEFT JOIN case_events ce ON
    	cae.case_id = ce.case_id
    LEFT JOIN detector_events de ON
    	ce.triggering_event_id = de.event_id
    WHERE
    	cae.customer_action_type = 'NOT_A_COMPROMISE'
    GROUP BY
    	detector_id
)

,
detector_precision AS (
SELECT
		COALESCE (tp.detector_id,
	fp.detector_id) AS detector_id,
		COALESCE (true_positives,
	0) AS true_positives,
		COALESCE (false_positives,
	0) AS false_positives,
		CASE 
			WHEN COALESCE (true_positives,
		0) + COALESCE (false_positives,
		0) = 0 THEN NULL
		WHEN COALESCE (true_positives,
		0) = 0 THEN 0
		ELSE round(COALESCE (true_positives,
		0)::NUMERIC /(COALESCE (true_positives,
		0) + COALESCE (false_positives,
		0)),
		2)
	END AS detector_precision
FROM
	true_positives tp
FULL JOIN false_positives fp ON
	tp.detector_id = fp.detector_id
)
"""

sql = ctes + """
SELECT 
	detector_id,
	detector_precision
FROM
	detector_precision
"""

run_select(sql)

,detector_id,detector_precision
0,DETECTOR_4,1.00
1,DETECTOR_3,0.63
2,DETECTOR_6,0.67
3,DETECTOR_1,0.67
4,DETECTOR_2,0.83
5,DETECTOR_5,0.70


## Which detector has the highest precision?
* I decided to use rank() to determine which detector has the highest precision. This means if there is a tie for rank 1 then all detectors with rank 1 will be in the results. I am assuming this is what end users expect.
* Otherwise, I'd not use rank() and just order the results by detector_precision and limit 1.

In [11]:
ctes = ctes + """
,
detector_rankings AS (
SELECT
	detector_id,
	detector_precision,
	RANK() OVER (
	ORDER BY detector_precision DESC) AS precision_rank
FROM
	detector_precision
GROUP BY
	detector_id,
	detector_precision
)
"""
sql = ctes + """
SELECT
	detector_id AS detectors_with_highest_precision,
	detector_precision
FROM
	detector_rankings
WHERE
	precision_rank = 1
"""
run_select(sql)

,detectors_with_highest_precision,detector_precision
0,DETECTOR_4,1.00


## Which detector has the lowest precision?

In [12]:
sql = ctes + """
SELECT 
		detector_id AS detectors_with_lowest_precision,
		detector_precision
FROM
	detector_rankings de
INNER JOIN (
	SELECT
		max(precision_rank) AS max_precision_rank
	FROM
		detector_rankings
		) mpr ON
	de.precision_rank = mpr.max_precision_rank
"""


run_select(sql)

,detectors_with_lowest_precision,detector_precision
0,DETECTOR_3,0.63


## 2.e. Which detector flags the highest number of customer-confirmed true positive cases?

In [13]:
sql = ctes + """
,
true_positive_count_rank AS (
SELECT
		detector_id,
		true_positives,
		RANK() OVER (
	ORDER BY true_positives DESC) AS count_rank
FROM
	true_positives
)

SELECT
	detector_id,
	true_positives
FROM
	true_positive_count_rank
WHERE
	count_rank = 1
"""
run_select(sql)


,detector_id,true_positives
0,DETECTOR_5,16


# 3. Building a data pipeline
* For this end-user table, I selected a **user event** as the primary business process. This is because user events are what trigger cases or are referred to by customers who report cases. Also, user events are what the ATO product uses to detect breaches.
* I selected the grain of this table to be an **event**; i.e. an `event_id`. This means each row in the end-user table will represent a distinct `event_id`.

Since I already have the tables in dbt, I can build a [model](https://github.com/danbratton/abnormal_exercise/blob/main/dbt/models/abnormal.sql).
* I'm assuming end-users only want the latest customer action in the model. The latest acton is what is needed to know if/how a customer resolved a case.
* I could implement other CTEs to get only the latest rows from other tables (such as `case_action_events`); however, my test case for unique on `event_id` (see below) passes with just this single CTE so I decided not to implement any more for this exercise. I can implement it at a later time if the test fails.
    * For example, if in the future it is possible for multiple detectors to flag the same `event_id` this model would need to be adapted. If this happened, the test below would fail so I would be able to adjust the model before it deploys.
```
WITH latest_customer_action_events AS (
SELECT
	DISTINCT ON
	(case_id)
		case_id,
		customer_action_ts AS latest_customer_action_ts,
		customer_action_type AS latest_customer_action_type,
	customer_comment AS latest_customer_comment
FROM
	{{ ref('customer_action_events') }} cae
ORDER BY
	case_id ASC,
	customer_action_ts DESC
)

SELECT
	ue.event_id,
	ue.customer_id,
	ue.user_id,
	ue.event_type,
	ue.event_timestamp,
	c.customer_name,
	c.cnt_users,
	c.remediation_action,
	de.detector_id,
	de.confidence_level AS detector_confidence_level,
	ce.case_id,
	ce.triggering_event_id,
	cce.reported_case_id,
	cce.report_timestamp,
	cae.action_id,
	cae.action_type,
	cae.action_timestamp,
	cae2.latest_customer_action_type,
	cae2.latest_customer_action_ts,
	cae2.latest_customer_comment
FROM
	{{ ref('user_events') }} ue
LEFT JOIN {{ ref('customers') }} c ON
	ue.customer_id = c.customer_id
LEFT JOIN {{ ref('detector_events') }} de ON
	ue.event_id = de.event_id
LEFT JOIN {{ ref('case_events') }} ce ON
	ue.event_id = ce.triggering_event_id
LEFT JOIN {{ ref('customer_case_events') }} cce ON
	ue.event_id = cce.reported_event_id
LEFT JOIN {{ ref('case_action_events') }} cae ON
	ce.case_id = cae.case_id
LEFT JOIN latest_customer_action_events cae2 ON
	ce.case_id = cae2.case_id
ORDER BY
	ue.event_id ASC
```

Also since this is in dbt, I can implement a test to assert that `event_id` is unique in the result in the [model_tests.yml](https://github.com/danbratton/abnormal_exercise/blob/main/dbt/models/model_tests.yml) file. I also added a test to assert all `event_id`s are not null.
```
-- models/model_tests.yml
models:
  - name: abnormal
    columns:
    - name: event_id
      tests:
        - unique
        - not_null
```
* There are other tests I could implement such as:
    * asserting all `event_id`s from `user_events` are in the model
    * asserting latest_customer_action_type are in a list of accepted values
* I decided to keep it lightweight for this exercise.

* After I run `dbt build` with this model, I can query it.

In [14]:
sql = """
SELECT 
    * 
FROM
    abnormal
"""
run_select(sql)

,event_id,customer_id,user_id,event_type,event_timestamp,customer_name,cnt_users,remediation_action,detector_id,detector_confidence_level,case_id,triggering_event_id,reported_case_id,report_timestamp,action_id,action_type,action_timestamp,latest_customer_action_type,latest_customer_action_ts,latest_customer_comment
0,1000,3,323,DATA_DOWNLOAD,2020-11-20 10:33:50,Lumina Dynamics Inc.,371,REVOKE_SESSIONS,None,None,NaN,NaN,NaN,NaT,NaN,None,NaT,None,NaT,None
1,1001,7,255,MAILFILTER,2020-12-09 10:10:21,Phoenix Crest Construction,347,REVOKE_SESSIONS,None,None,NaN,NaN,NaN,NaT,NaN,None,NaT,None,NaT,None
2,1004,3,190,SIGN_IN,2020-02-12 03:27:06,Lumina Dynamics Inc.,371,REVOKE_SESSIONS,None,None,NaN,NaN,NaN,NaT,NaN,None,NaT,None,NaT,None
3,1005,10,12,DATA_DOWNLOAD,2020-03-30 08:21:36,Aurora Axis Enterprises,257,REVOKE_SESSIONS_RESET_PASSWORD,None,None,NaN,NaN,NaN,NaT,NaN,None,NaT,None,NaT,None
4,1008,8,182,POSTURE_CHANGE,2020-01-12 23:25:23,"Nexus Auto Parts, Inc",200,REVOKE_SESSIONS,None,None,NaN,NaN,NaN,NaT,NaN,None,NaT,None,NaT,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,9988,8,183,SIGN_IN,2020-08-12 01:40:57,"Nexus Auto Parts, Inc",200,REVOKE_SESSIONS,None,None,NaN,NaN,NaN,NaT,NaN,None,NaT,None,NaT,None
4996,9989,6,137,MAILFILTER,2020-01-30 11:19:47,Elysian Capital,211,NOTIFICATION,None,None,NaN,NaN,NaN,NaT,NaN,None,NaT,None,NaT,None
4997,9994,4,246,SIGN_IN,2020-11-18 11:45:57,Titan Forge Technologies,248,REVOKE_SESSIONS_RESET_PASSWORD,DETECTOR_1,LOW,NaN,NaN,NaN,NaT,NaN,None,NaT,None,NaT,None
4998,9997,7,87,MAILFILTER,2020-01-12 01:58:59,Phoenix Crest Construction,347,REVOKE_SESSIONS,None,None,NaN,NaN,NaN,NaT,NaN,None,NaT,None,NaT,None
